In [ ]:
import cv2
import dlib
import imutils
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist
import time
import winsound
import base64
import jwt
import os
import requests


from flask import Flask,flash, render_template, request, redirect, url_for, jsonify, session

Url= 'https://peach-violet-rhinoceros.glitch.me/api/v1'
studentUrl=Url+"/student/login"
joinSessionUrl=Url+"/room/"

app = Flask(__name__, template_folder='templates', static_folder='static')

app.secret_key = 'session'


@app.route('/video_frames', methods=['POST'])
def process_video_frames():
    if request.method == 'POST':
        print("HI")
    # Access the video frames in the request body and pass them to your attention level detection function
        video_frames = request.data
        run_eye_tracking(video_frames)
#     return attention_level

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password'] 
        data = {'emailID': username, 'password': password}
        response = requests.post(studentUrl, json=data)
        if response.json()['success']:
            # If the authentication was successful, store the token in the user's session
            token = response.json()['data']['tokem']
            session['user_data'] = {
                'token': token,
                'student': {
                    '_id': response.json()['data']['student']['_id'],
                    'emailID': response.json()['data']['student']['emailID'],
                    'emailVerified': response.json()['data']['student']['emailVerified'] ,
                }
            }
#             print(session)
            print("User Logged in Successfully")
            return redirect(url_for('join_meeting'))
        else:
            session.clear()
            print("Invalid Login Details")       
    else:
        return render_template('login.html')
    
def getRoomId(roomId):
    return roomId
            

        
        
#@app.route('/video_feed')
#def video_feed():
    #cap = cv2.VideoCapture(0)
    #return Response(generate_frames(cap), mimetype='multipart/x-mixed-replace; boundary=frame')


#def generate_frames(cap):
#     while True:
#         success, frame = cap.read()
#         if not success:
#             break
#         else:
#             ret, buffer = cv2.imencode('.jpg', frame)
#             frame = buffer.tobytes()
#             yield (b'--frame\r\n'
#                    b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
            

@app.route('/meeting_room', methods=['GET', 'POST'])
def meeting_room():
    if request.method == 'POST':
        print("Hi")
        #run_eye_tracking()
        return render_template('meeting_room.html')

@app.route('/room/', methods=['GET', 'POST'])
def join_meeting():
    room_id="abcd";
    room=getRoomId(room_id)
#     print(room)
    
    if request.method == 'POST':
        if "user_data" in session:
            print("Hello, logged in user!")
        else:
            print("Please log in")
            return render_template('login.html')
            
        session_id=room
        roomlink=joinSessionUrl+session_id
        user_data = session['user_data']
        response=requests.patch(roomlink,  headers={
               'Authorization': user_data['token']})
#         print(roomlink)
#         print(response.json()['success'])
        # session ID matches, redirect to another page   
        return render_template('meeting_room.html')
    else:
        print("False Info")
        # session ID doesn't match, display error message
        return render_template('join_meeting.html', error='Session ID is incorrect')
        #return render_template('join_meeting.html')
        





@app.route('/run_eye_tracking')
def run_eye_tracking(video_frames):
    def play_alarm():
        #time.sleep(3)
        frequency = 3500  # Set Frequency To 2500 Hertz
        duration = 5000  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
   

    # Define a function to calculate the eye aspect ratio
    def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear


    
    # Initialize dlib's face detector and create a dictionary to map facial landmark indexes to face parts
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    face_parts = face_utils.FACIAL_LANDMARKS_IDXS
    (left_eye_start, left_eye_end) = face_parts["left_eye"]
    (right_eye_start, right_eye_end) = face_parts["right_eye"]

    # Start the video stream and wait for the camera to warm up
    with open('video.mp4', 'wb') as f:
        f.write(base64.b64decode(video_frames))
        
    video_path = os.path.join('videos', 'video.mp4')
        
    vs = video_path
#     time.sleep(2.0)

    # Initialize variables for calculating the attention level
    drowsy_counter = 0
    drowsy_threshold = 15
    focused_counter = 0
    focused_threshold = 5
    unfocused_counter = 0
    unfocused_threshold = 20
    max_attention_level = "unfocused"
    max_attention_time = time.time()
    
    eyes_closed = False

    # Loop over frames from the video stream
    while True:
        ret, frame = vs.read()
        if not ret:
            continue

        # Resize the frame
        frame = imutils.resize(frame, width=800)
        
        
    
        # Convert the frame to grayscale and detect faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
    
        # Reset the eye positions
        left_eye_position = None
        right_eye_position = None

        # Loop over the face detections
        for rect in rects:
            # Determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
    
            # Extract the left and right eye coordinates and compute the eye aspect ratio for each eye
            left_eye = shape[left_eye_start:left_eye_end]
            right_eye = shape[right_eye_start:right_eye_end]
            left_eye_ear = eye_aspect_ratio(left_eye)
            right_eye_ear = eye_aspect_ratio(right_eye)

            # Compute the average eye aspect ratio
            avg_ear = (left_eye_ear + right_eye_ear) / 2.0
    
            # Check if the average eye aspect ratio is below the drowsy threshold
            if avg_ear < 0.25:
                drowsy_counter += 1
                focused_counter = 0
                unfocused_counter = 0
            else:
                drowsy_counter = 0
                focused_counter += 1
                # Check if eyes are not in the frame
                if left_eye.shape[0] == 0 or right_eye.shape[0] == 0:
                    unfocused_counter += 1
                else:
                    left_eye_position = left_eye
                    right_eye_position = right_eye

            # Check if the attention level is drowsy
            if drowsy_counter >= drowsy_threshold:
                max_attention_level = "drowsy"
                if drowsy_counter == drowsy_threshold:
                    alarm_start_time = time.time()
                if time.time() - alarm_start_time >= 3 and not eyes_closed:
                    play_alarm()
                    #eyes_closed = True
                    drowsy_counter = 0
            
            # Check if the attention level is unfocused
            if unfocused_counter >= unfocused_threshold:
                max_attention_level = "unattentive"

            # Check if the attention level is focused
            if focused_counter >= focused_threshold:
                max_attention_level = "focused"

	  

     # Update the max attention level
        if time.time() - max_attention_time >= 10:
            print(f"Max attention level for the past 10 Seconds : {max_attention_level}")
            #generate_heatmap(max_attention_level)
            max_attention_level = "unattentive"
            max_attention_time = time.time()

        # Display the frame
#         cv2.imshow("Student Attention Dettection", frame)
#         key = cv2.waitKey(1) & 0xFF

        # If the `q` key was pressed, break from the loop
#         if key == ord("q"):
#             break

    # Cleanupq
    cv2.destroyAllWindows()
    
# @app.route('/video_feed')
# def video_feed():
#     return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')
    
def generate_heatmap(attention_level):
    global heatmap
    color_map = {"focused": (0, 255, 0), "unfocused": (0, 0, 255), "drowsy": (255, 0, 0), "unattentive": (255, 255, 0)}

    color = color_map[attention_level]
    center_pixel = (heatmap.shape[1] // 2, heatmap.shape[0] // 2)

    heatmap[center_pixel] = tuple(color)
    print(heatmap)

    

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2023 22:28:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2023 22:28:20] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [14/Apr/2023 22:28:20] "GET /room/ HTTP/1.1" 200 -


User Logged in Successfully
False Info
Hello, logged in user!


127.0.0.1 - - [14/Apr/2023 22:28:34] "POST /room/ HTTP/1.1" 200 -
